# imports

In [1]:
import numpy as np
from math import sqrt
from pprint import pprint
from sklearn import datasets, linear_model, metrics
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split, cross_val_score
from sklearn import preprocessing
from sklearn.metrics import make_scorer, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate

# datasets

In [6]:
import os
import pandas as pd
path = 'C:/Users/plane/OneDrive/Escritorio/COMPUTING SCIENCE/MASTER Inteligencia artificial/05_Aprendizaje_supervisado/data/BostonHousing.csv'

data = pd.read_csv(path, sep= ',' )
print(data.shape)
data.sample(5)

(506, 14)


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
399,9.91655,0.0,18.10,0,0.693,5.852,77.8,1.5004,24,666,20.2,338.16,29.97,6.3
479,14.33370,0.0,18.10,0,0.614,6.229,88.0,1.9512,24,666,20.2,383.32,13.11,21.4
126,0.38735,0.0,25.65,0,0.581,5.613,95.6,1.7572,2,188,19.1,359.29,27.26,15.7
478,10.23300,0.0,18.10,0,0.614,6.185,96.7,2.1705,24,666,20.2,379.70,18.03,14.6
333,0.05083,0.0,5.19,0,0.515,6.316,38.1,6.4584,5,224,20.2,389.71,5.68,22.2


# split hold out y escalado

In [23]:
X = data.drop('medv', axis= 1)
y = data['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

scaler = preprocessing.StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

metricas = {
  'MAE' : lambda y, y_pred: metrics.mean_absolute_error(y, y_pred),
  'RMSE': lambda y, y_pred: sqrt(metrics.mean_squared_error(y, y_pred)),
  'MAPE': lambda y, y_pred: np.mean(np.abs((y - y_pred) / y)) * 100,
  'R2'  : lambda y, y_pred: metrics.r2_score(y, y_pred)}

(404, 13)
(102, 13)
(404,)
(102,)


# ELASTIC NET

sklearn.linear_model.ElasticNet(alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')



- **alphafloat**, default=1.0

    Constant that multiplies the penalty terms. Defaults to 1.0. See the notes for the exact mathematical meaning of this       parameter. alpha = 0 is equivalent to an ordinary least square, solved by the LinearRegression object. For numerical       reasons, using alpha = 0 with the Lasso object is not advised. Given this, you should use the LinearRegression object.
    l1_ratiofloat, default=0.5

- **l1_ratio**, defautl = 0.5

    The ElasticNet mixing parameter, with 0 <= l1_ratio <= 1. For l1_ratio = 0 the penalty is an L2 penalty. For l1_ratio =     1   it is an L1 penalty. For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.


In [24]:
elastic = linear_model.ElasticNet(fit_intercept=True) 

h_params = {'l1_ratio': [0.2, 0.4, 0.6, 0.8, 1],
            'alpha': [0.01, 0.1, 1, 10, 100],
            }

GS_elas = GridSearchCV(estimator=elastic,                                     # el modelo
                       param_grid=h_params,                                   # red de hiperparametros
                       scoring='r2',                                          # metricas
                       cv= KFold(n_splits=5, shuffle=True, random_state=42),  # cross validation
                       #refit= 'R2',                                          # en caso de más de una metrica hay que indicar cual es la de referencia
                       n_jobs= -1)


In [25]:
GS_elas.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1, 10, 100],
                         'l1_ratio': [0.2, 0.4, 0.6, 0.8, 1]},
             scoring='r2')

In [26]:
print(GS_elas.best_estimator_)
print(GS_elas.best_score_)
best_ELASTINET = GS_elas.best_estimator_

ElasticNet(alpha=0.01, l1_ratio=0.2)
0.7189606836262387


In [27]:
y_pred = best_ELASTINET.predict(X_test)

In [30]:
for nombre, metrica in metricas.items():
    print(f'para la metrica \t {nombre} \t  se obtienen un valor de \t  ---> \t {metrica(y_test, y_pred)}')

para la metrica 	 MAE 	  se obtienen un valor de 	  ---> 	 3.1797770819670044
para la metrica 	 RMSE 	  se obtienen un valor de 	  ---> 	 4.936618013671427
para la metrica 	 MAPE 	  se obtienen un valor de 	  ---> 	 16.833308685750247
para la metrica 	 R2 	  se obtienen un valor de 	  ---> 	 0.6676811646328558
